<a href="https://colab.research.google.com/github/Shibu4064/EMNLP/blob/main/classification_upscaled_with_buetcsenlp_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets
!pip install transformers
!pip install transformers[torch]
!pip install transformers[tf-cpu]
!pip install transformers[flax]
!pip install torch
!pip install tensorflow
!pip install evaluate
!pip install torchsummary
!pip install gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.6/225.6 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.2/451.2 kB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import multiprocessing
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import transformers
import torch.nn as nn
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoModelForMaskedLM, AutoModelForPreTraining, EarlyStoppingCallback, AutoModelForSequenceClassification, BertForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from transformers import BertForMaskedLM, DistilBertForMaskedLM
from transformers import BertTokenizer, DistilBertTokenizer
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from tokenizers import BertWordPieceTokenizer

In [ ]:
# torch.set_default_device('cuda')

In [4]:
import gdown

dataset_link = "https://drive.google.com/uc?id=1FoyySUxspmkT4KNwSsYWDqw8TVUF-meV"

gdown.download(dataset_link,"checkpoint.zip")



Downloading...
From: https://drive.google.com/uc?id=1FoyySUxspmkT4KNwSsYWDqw8TVUF-meV
To: /content/checkpoint.zip
100%|██████████| 1.86G/1.86G [00:15<00:00, 121MB/s]


'checkpoint.zip'

In [5]:
!unzip checkpoint.zip

Archive:  checkpoint.zip
   creating: content/tmp_trainer/checkpoint-7500/
  inflating: content/tmp_trainer/checkpoint-7500/special_tokens_map.json  
  inflating: content/tmp_trainer/checkpoint-7500/generation_config.json  
  inflating: content/tmp_trainer/checkpoint-7500/rng_state.pth  
  inflating: content/tmp_trainer/checkpoint-7500/vocab.txt  
  inflating: content/tmp_trainer/checkpoint-7500/tokenizer.json  
  inflating: content/tmp_trainer/checkpoint-7500/config.json  
  inflating: content/tmp_trainer/checkpoint-7500/trainer_state.json  
  inflating: content/tmp_trainer/checkpoint-7500/pytorch_model.bin  
  inflating: content/tmp_trainer/checkpoint-7500/optimizer.pt  
  inflating: content/tmp_trainer/checkpoint-7500/training_args.bin  
  inflating: content/tmp_trainer/checkpoint-7500/tokenizer_config.json  
  inflating: content/tmp_trainer/checkpoint-7500/scheduler.pt  


In [26]:
from transformers import AutoTokenizer, AutoModel, AutoConfig

model_name = "csebuetnlp/banglabert"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# config = AutoConfig.from_pretrained("/content/content/tmp_trainer/checkpoint-7500/config.json")

# model = AutoModel.from_pretrained("/content/content/tmp_trainer/checkpoint-7500/pytorch_model.bin",config=config)
classifier = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=3)

classifier

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [7]:
# bangla_bert = AutoModelForMaskedLM.from_pretrained("")

# tokenizer_name = 'sagorsarker/bangla-bert-base'

# tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, use_fast=True)

train_texts = ["আমার কিছু ভালো লাগে না","আমার সব কিছুই ভালো লাগে"]
train_encodings = tokenizer(train_texts, truncation=True, padding=True)

# train_encodings["input_ids"]

# print(train_encodings['input_ids'])

output = classifier(torch.tensor(train_encodings["input_ids"]))
# print(output[0].shape)
print(output[0].shape)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


torch.Size([2, 3])


In [15]:
import gdown

train_data_link = "https://drive.google.com/uc?id=1mAlatqgef3Jy64ZmHGJ9vNBl3Voxsm21"
gdown.download(train_data_link,"train.csv")

val_data_link =   "https://drive.google.com/uc?id=1yWabvxdMa6GWaE_K6cVNI0TylCP3MYHg"
gdown.download(val_data_link,"val.csv")

# fine_tuned_model_path = "https://drive.google.com/uc?id=1VchxbcjhTE4G0AhiJuFbvO_Mnfwhp1Mp"
# gdown.download(fine_tuned_model_path,"fine_tuned_bert.pth")

Downloading...
From: https://drive.google.com/uc?id=1mAlatqgef3Jy64ZmHGJ9vNBl3Voxsm21
To: /content/train.csv
100%|██████████| 1.12M/1.12M [00:00<00:00, 81.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yWabvxdMa6GWaE_K6cVNI0TylCP3MYHg
To: /content/val.csv
100%|██████████| 345k/345k [00:00<00:00, 58.2MB/s]


'val.csv'

In [16]:
train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("val.csv")

In [17]:
train_df.head()

,text,label
0,"সকল গনমাধ্যম এখন আওমীলীগের চামচা গোলাম,,কুকুরে...",1
1,মিথ্যা কথা আর কত বলবে।,0
2,যমুনাকে ধন্যবাদ,0
3,টাকলু র দায় সব কিছু,1
4,একটা ভুয়া ভিডিও তৈরী করে মানুষের চোখে ধুলো দেও...,1


In [18]:
from sklearn.model_selection import train_test_split
def pandaToList(dataset):
    text_list = []
    label_list = []
    for i in range(len(dataset['text'])):
      text_list.append(dataset['text'][i])
      label_list.append(int(dataset['label'][i]))
    return text_list, label_list

train_texts, train_labels =  pandaToList(train_df)
val_texts, val_labels = pandaToList(val_df)

In [19]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [ ]:
# print(train_encodings['input_ids'][0])
# print(val_encodings["input_ids"][0])

In [20]:
import torch

class ViolenceDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ViolenceDataset(train_encodings, train_labels,)
val_dataset = ViolenceDataset(val_encodings, val_labels)

In [ ]:
# print(train_dataset[0])
# print(val_dataset[0])

In [21]:


from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='macro')
    precision = precision_score(y_true=labels, y_pred=pred, average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='macro')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}



In [29]:
from transformers import  Trainer, TrainingArguments, EarlyStoppingCallback, IntervalStrategy, AutoModelForSequenceClassification

training_args = TrainingArguments(
   f"training_with_callbacks",
   evaluation_strategy = IntervalStrategy.STEPS, # "steps"
   #output_dir= output_dir,
   eval_steps = 250, # Evaluation and Save happens every 250 steps
   save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted.

   learning_rate=2e-5,
   per_device_train_batch_size=1,
   per_device_eval_batch_size=1,
   num_train_epochs=50,
   weight_decay=0.01,
   metric_for_best_model = 'f1',
  #  generator=torch.Generator(device='cuda'),
   load_best_model_at_end=True)

trainer = Trainer(
    model=classifier,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    # generator=torch.Generator(device='cuda'),
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)]
)
# trainer.to('cuda')

In [30]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
250,No log,1.042449,0.356391,0.671650,0.453024,0.286639
500,1.035900,0.680417,0.744361,0.759909,0.654780,0.686461
750,1.035900,1.144352,0.734586,0.719725,0.771539,0.725716
1000,1.134200,0.970263,0.772932,0.742947,0.767301,0.753613
1250,1.134200,1.197777,0.765414,0.729262,0.762142,0.740604
1500,1.124300,1.214084,0.741353,0.730128,0.763031,0.731653
1750,1.124300,1.803447,0.631579,0.659510,0.695596,0.615865
2000,1.123000,1.163490,0.765414,0.751864,0.753350,0.747364
2250,1.123000,1.118202,0.772180,0.745415,0.767273,0.754915
2500,1.110400,1.203711,0.775940,0.753240,0.765369,0.755774


TrainOutput(global_step=12250, training_loss=0.6626583912129305, metrics={'train_runtime': 2200.532, 'train_samples_per_second': 94.682, 'train_steps_per_second': 94.682, 'total_flos': 982050275934000.0, 'train_loss': 0.6626583912129305, 'epoch': 2.94})

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [32]:
trainer.save_model("/content/drive/MyDrive/EMNLP/buetcsenlp_banglabert_classifierpth")